In [1]:
import numpy as np
import math 
import queue
import os
import neat
import visualize
import random

import GameManager
from GameManager import map_manager, Tile, Unit, Team

import Neat_AI
from Neat_AI import eval_genomes, run

In [43]:
from importlib import reload
reload(GameManager)
reload(Neat_AI)

from GameManager import map_manager, Tile, Unit, Team
from Neat_AI import eval_genomes, run

In [3]:
local_dir = os.path.abspath('')
config_path = os.path.join(local_dir, 'config-feedforward')

In [4]:
dimensions = (5,5)
MapManager = map_manager(dimensions)

pos1 = (0, 0)
pos2 = (4, 4)
unit1 = MapManager.place_unit(pos1, 0)
unit2 = MapManager.place_unit(pos2, 1)

print(MapManager)

MapManager.find_movement(unit1)
MapManager.move_unit(unit1, (1, 1))

MapManager.find_movement(unit1)
MapManager.move_unit(unit1, (1, 1))
print(MapManager)

[0][ ][ ][ ][ ]
[ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ]
[ ][ ][ ][ ][1]

[ ][ ][ ][ ][ ]
[ ][0][ ][ ][ ]
[ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ]
[ ][ ][ ][ ][1]



In [5]:
dimensions = (5,5)
move_dist = 4

pos_pick = random.randint(0, 3)
pos_1 = (0, 0)
pos_2 = (0, 0)
if (pos_pick == 0):
    pos_1 = (0, 0)
    pos_2 = (4, 4)
elif (pos_pick == 1):
    pos_1 = (4, 0)
    pos_2 = (0, 4)
elif (pos_pick == 2):
    pos_1 = (0, 4)
    pos_2 = (4, 0)
elif(pos_pick == 3):
    pos_1 = (4, 4)
    pos_2 = (0, 0)

#dist_list = []
turns = 0
while (dist > 0.0 and turns < 2): #Turn Count limit may have to be modified
    print("unit 1 pos: " + str(pos1))
    print("unit 2 pos: " + str(pos2))

    input_list = []
    input_list.append(pos1[0]/(dimensions[0]-1))
    input_list.append(pos1[1]/(dimensions[1]-1))

    input_list.append(pos2[0]/(dimensions[0]-1))
    input_list.append(pos2[1]/(dimensions[1]-1))

    print("input pos: " + str(input_list))

    input_tup = tuple(input_list)
    format_in = [ '%.2f' % elem for elem in input_tup ]
    print("input vector: " + str(format_in))

    #use output dimensions
    output = saved_net.activate(input_tup)

    format_out = [ '%.2f' % elem for elem in output ]
    print("output vector: " + str(format_out))

    move = [round(output[0] * move_dist), round(output[1] * move_dist)]
    #move = (max(min(round(output[0] * unit1.max_move), dimensions[0]-1-unit1.pos[0]), -unit1.pos[0]),
    #        max(min(round(output[1] * unit1.max_move), dimensions[1]-1-unit1.pos[1]), -unit1.pos[1]))
    print("move: " + str(move))
    new_pos = list(np.add(pos1, move))
    if (new_pos[0] >= dimensions[0]):
        new_pos[0] = dimensions[0]-1
    if (new_pos[1] >= dimensions[1]):
        new_pos[1] = dimensions[1]-1

    if (new_pos[0] < 0):
        new_pos[0] = 0
    if (new_pos[1] < 0):
        new_pos[1] = 0
    pos1 = tuple(new_pos)

    print("new_pos: "+ str(pos1))
    turns += 1
print("fitness: " + str(5.656854249492381 - math.dist(pos1, pos2)))

NameError: name 'dist' is not defined

In [ ]:
win_net = run(config_path)

	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_conn


 ****** Running generation 0 ****** 

Population's average fitness: 0.21667 stdev: 0.16017
Best fitness: 0.70000 - size: (3, 4) - species 1 - id 8
Average adjusted fitness: 0.240
Mean genetic distance 2.486, standard deviation 0.822
Population of 150 members in 7 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0    51      0.7    0.203     0
     2    0     7      0.5    0.208     0
     3    0    10      0.4    0.211     0
     4    0    20      0.6    0.225     0
     5    0    44      0.5    0.231     0
     6    0    10      0.3    0.300     0
     7    0     8      0.3    0.300     0
Total extinctions: 0
Generation time: 1.627 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.19800 stdev: 0.20477
Best fitness: 0.80000 - size: (3, 4) - species 5 - id 35
Average adjusted fitness: 0.175
Mean genetic distance 2.135, standard deviation 0.791
Population of 150 members in 7 species:
   ID   age  size  fitnes

Population's average fitness: 0.31333 stdev: 0.27097
Best fitness: 1.00000 - size: (4, 6) - species 3 - id 1618
Average adjusted fitness: 0.233
Mean genetic distance 2.250, standard deviation 0.683
Population of 149 members in 7 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   12    10      0.4    0.117     3
     2   12    20      0.6    0.250     3
     3   12     1      1.0    0.379     0
     4   12    61      0.4    0.160     5
     5   12     9      0.3    0.075     2
     6   12    22      0.7    0.306     3
     7   12    26      0.8    0.345     6
Total extinctions: 0
Generation time: 1.425 sec (1.440 average)

 ****** Running generation 13 ****** 

Population's average fitness: 0.31141 stdev: 0.32882
Best fitness: 1.00000 - size: (4, 6) - species 4 - id 1618
Average adjusted fitness: 0.215
Mean genetic distance 2.339, standard deviation 0.633
Population of 151 members in 7 species:
   ID   age  size  fitness  adj fit  stag


Population's average fitness: 0.34966 stdev: 0.32700
Best fitness: 1.00000 - size: (4, 4) - species 1 - id 3197
Average adjusted fitness: 0.345
Mean genetic distance 2.327, standard deviation 0.765
Population of 150 members in 8 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   24    18      1.0    0.377    15
     2   24    17      0.7    0.319    10
     3   24    12      0.6    0.181    12
     4   24     7      0.7    0.292    11
     5   24    27      1.0    0.235    14
     6   24    12      0.7    0.272     5
     7   24     5      1.0    0.575     7
     8    1    52      1.0    0.512     0
Total extinctions: 0
Generation time: 1.323 sec (1.362 average)
Saving checkpoint to neat-checkpoint-24

 ****** Running generation 25 ****** 

Population's average fitness: 0.39667 stdev: 0.35579
Best fitness: 1.00000 - size: (3, 4) - species 5 - id 3235
Average adjusted fitness: 0.355
Mean genetic distance 2.281, standard deviation 0.748


Population's average fitness: 0.39865 stdev: 0.36205
Best fitness: 1.00000 - size: (6, 7) - species 7 - id 4791
Average adjusted fitness: 0.363
Mean genetic distance 2.311, standard deviation 0.636
Population of 149 members in 8 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   35    31      1.0    0.275    26
     2   35    24      1.0    0.464    21
     3   35     4      1.0    0.528    23
     4   35    27      1.0    0.343    22
     5   35     9      0.7    0.133    25
     6   35    16      0.7    0.288    16
     7   35    14      1.0    0.330    18
     8   12    24      1.0    0.541    11
Total extinctions: 0
Generation time: 1.393 sec (1.415 average)

 ****** Running generation 36 ****** 

Population's average fitness: 0.44161 stdev: 0.34429
Best fitness: 1.00000 - size: (5, 7) - species 1 - id 4838
Average adjusted fitness: 0.400
Mean genetic distance 2.340, standard deviation 0.605
Population of 151 members in 8 species:


Population's average fitness: 0.35533 stdev: 0.38219
Best fitness: 1.00000 - size: (3, 4) - species 4 - id 6159
Average adjusted fitness: 0.339
Mean genetic distance 2.169, standard deviation 0.768
Population of 151 members in 8 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   46    17      1.0    0.321    37
     2   46    19      1.0    0.322    32
     3   46    21      1.0    0.286    34
     4   46    24      1.0    0.457    33
     5   46     8      0.3    0.120    36
     6   46    15      0.7    0.315    27
     7   46    23      1.0    0.414    29
     8   23    24      1.0    0.476    22
Total extinctions: 0
Generation time: 1.477 sec (1.361 average)

 ****** Running generation 47 ****** 

Population's average fitness: 0.44636 stdev: 0.37870
Best fitness: 1.00000 - size: (3, 4) - species 3 - id 3235
Average adjusted fitness: 0.431
Mean genetic distance 2.162, standard deviation 0.700
Population of 151 members in 8 species:


Population's average fitness: 0.45200 stdev: 0.40360
Best fitness: 1.00000 - size: (3, 3) - species 6 - id 7204
Average adjusted fitness: 0.463
Mean genetic distance 1.878, standard deviation 0.711
Population of 151 members in 8 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   57    16      1.0    0.379    48
     2   57    21      1.0    0.505    43
     3   57    20      1.0    0.553    45
     4   57    14      1.0    0.600    44
     5   57    14      1.0    0.384    47
     6   57    32      1.0    0.527     4
     7   57    19      1.0    0.450    40
     8   34    15      1.0    0.309    33
Total extinctions: 0
Generation time: 1.293 sec (1.349 average)

 ****** Running generation 58 ****** 

Population's average fitness: 0.43046 stdev: 0.41126
Best fitness: 1.00000 - size: (3, 3) - species 6 - id 7204
Average adjusted fitness: 0.439
Mean genetic distance 1.860, standard deviation 0.724
Population of 147 members in 8 species:


Population's average fitness: 0.46933 stdev: 0.40397
Best fitness: 1.00000 - size: (3, 3) - species 6 - id 7204
Average adjusted fitness: 0.478
Mean genetic distance 1.910, standard deviation 0.699
Population of 153 members in 8 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   68    15      1.0    0.437    59
     2   68    17      1.0    0.394    54
     3   68    11      1.0    0.520    56
     4   68    24      1.0    0.467    55
     5   68    16      1.0    0.590    58
     6   68    31      1.0    0.453    15
     7   68    21      1.0    0.530    51
     8   45    18      1.0    0.435    44
Total extinctions: 0
Generation time: 1.260 sec (1.277 average)

 ****** Running generation 69 ****** 

Population's average fitness: 0.40458 stdev: 0.38929
Best fitness: 1.00000 - size: (3, 3) - species 6 - id 9217
Average adjusted fitness: 0.402
Mean genetic distance 1.900, standard deviation 0.692
Population of 152 members in 8 species:


Population's average fitness: 0.46333 stdev: 0.42886
Best fitness: 1.00000 - size: (3, 3) - species 6 - id 9217
Average adjusted fitness: 0.463
Mean genetic distance 1.908, standard deviation 0.714
Population of 150 members in 8 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   79    19      1.0    0.533    70
     2   79    15      1.0    0.450    65
     3   79    11      1.0    0.316    67
     4   79    23      1.0    0.582    66
     5   79    13      1.0    0.420    69
     6   79    31      1.0    0.443    26
     7   79    19      1.0    0.450    62
     8   56    19      1.0    0.506    55
Total extinctions: 0
Generation time: 1.474 sec (1.341 average)
Saving checkpoint to neat-checkpoint-79

 ****** Running generation 80 ****** 

Population's average fitness: 0.43133 stdev: 0.40169
Best fitness: 1.00000 - size: (2, 2) - species 6 - id 10692
Average adjusted fitness: 0.445
Mean genetic distance 1.880, standard deviation 0.718

Population's average fitness: 0.47020 stdev: 0.40211
Best fitness: 1.00000 - size: (2, 2) - species 6 - id 10692
Average adjusted fitness: 0.487
Mean genetic distance 1.877, standard deviation 0.741
Population of 149 members in 8 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   90    13      1.0    0.525    81
     2   90    19      1.0    0.522    76
     3   90    20      1.0    0.353    78
     4   90    14      1.0    0.481    77
     5   90    16      1.0    0.362    80
     6   90    32      1.0    0.394    37
     7   90    20      1.0    0.553    73
     8   67    15      1.0    0.708    66
Total extinctions: 0
Generation time: 1.299 sec (1.348 average)

 ****** Running generation 91 ****** 

Population's average fitness: 0.43758 stdev: 0.39519
Best fitness: 1.00000 - size: (2, 2) - species 6 - id 10692
Average adjusted fitness: 0.451
Mean genetic distance 1.858, standard deviation 0.729
Population of 150 members in 8 species

In [21]:
saved_net = win_net